In [ ]:
#working script to do both rotations and mirroring and put image files in output_dir and add to csv
#multiplies original dataset by 4 to get 22370*4=89480 files
#to get more artifical test data rotate each photo 3 times to 90, 180, 270 non-randomized angles

import random
import pandas as pd
import os
from PIL import Image

#update these image directories to personal names
image_dir = "/home/jovyan/training_images"
output_dir = "/home/jovyan/prepro_images"
os.makedirs(output_dir, exist_ok=True)

def mirror_image(img, mode):
    if mode == "x":   # vertical flip
        return img.transpose(Image.FLIP_TOP_BOTTOM)
    if mode == "y":   # horizontal flip
        return img.transpose(Image.FLIP_LEFT_RIGHT)

def augment_with_rot_mir(input_csv, output_csv):
    df = pd.read_csv(input_csv)
    new_rows = []

    #rotating each image by a random increment of 90deg, output 1 image per 1 input
    #for _, row in df.iloc[0:10].iterrows():
    for _, row in df.iterrows():
        original_path = os.path.join(image_dir, row["relative_img_loc"])
        img = Image.open(original_path)
    
        base_name, ext = os.path.splitext(original_path)
        
        # build the output filename
        rotation_number = random.choice([90,180,270])
        new_filename = f"{os.path.splitext(os.path.basename(original_path))[0]}_{rotation_number}{ext}"
        save_path = os.path.join(output_dir, new_filename)

        #to do all 90deg incremented rotations and create more artificial data
        #for angle in [90,180,270]:
        #new_filename = f"{os.path.splitext(os.path.basename(original_path))[0]}_{angle}{ext}"
        #save_path = os.path.join(output_dir, new_filename)
        #rotated_img = img.rotate(angle, expand=True)
        #rotated_img.save(save_path)

        # save mirrored image
        rotated_img = img.rotate(rotation_number, expand=True)
        rotated_img.save(save_path)

        # duplicate metadata row
        new_row = row.copy()
        new_row["relative_img_loc"] = new_filename

        new_rows.append(new_row)

    #mirroring images over both x and y axes to output 2 new images per 1 input
    #for _, row in df.iloc[0:10].iterrows():
    for _, row in df.iterrows():
        original_path = os.path.join(image_dir, row["relative_img_loc"])
        img = Image.open(original_path)
    
        base, ext = os.path.splitext(original_path)

        for mode in ["x", "y"]:
            # build the output filename
            new_filename = f"{os.path.splitext(os.path.basename(original_path))[0]}_{mode}{ext}"
            save_path = os.path.join(output_dir, new_filename)

            # save mirrored image
            mirrored = mirror_image(img, mode)
            mirrored.save(save_path)

            # duplicate metadata row
            new_row = row.copy()
            new_row["relative_img_loc"] = new_filename

            new_rows.append(new_row)

    # combine original + new rows to outup prepro.csv
    df_aug = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    df_aug.to_csv(output_csv, index=False)

#change paths to your directorys, currently inputs from my hackdata and outputs to our cloned repo
augment_with_rot_mir("/home/jovyan/hackdata/sentinel-beetles/public_release/train.csv", "/home/jovyan/UCSD_Smash_NSF_HDR_ML_2025_Challenge/prepro.csv")
folder = "/home/jovyan/prepro_images"
files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
print("Huzzah!")
print("You have successfully created", len(files), "files. Good job :D")